In [ ]:
#Importing  necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, \
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


In [ ]:
#Loading  Dataset


In [ ]:
# Path to the Flickr8k dataset
dataset_text = r"/content/captions.txt"
dataset_images = r"C:\Users\k.s.sarath chandu\Downloads\Flickr_8k_Dataset\Images"

# Load the Flickr8k token file containing image filenames and captionstoken_file = os.path.join(dataset_text, 'captions.txt')
token_file = dataset_text

In [ ]:
# Preprocessing  Captions

In [ ]:
def load_captions(filename):
    # Load captions into a dictionary
    captions = {}
    with open(filename, 'r') as f:
        for line in f.readlines():
            tokens = line.split()
            image_id, image_caption = tokens[0], tokens[1:]
            image_id = image_id.split('.')[0]  # Remove the image file extension
            image_caption = ' '.join(image_caption)
            if image_id not in captions:
                captions[image_id] = []
            captions[image_id].append(image_caption)
    return captions

captions = load_captions(token_file)

In [ ]:
# Clean Captions
 #(load captions from the captions.txt file and
 #clean them by performing tasks like removing punctuation,
 #converting to lowercase, and removing non-alphabetic tokens.)

In [ ]:
def clean_captions(captions):
    table = str.maketrans('', '', string.punctuation)
    for key, caption_list in captions.items():
        for i in range(len(caption_list)):
            caption = caption_list[i]
            caption = caption.split()
            caption = [word.lower() for word in caption]
            caption = [word.translate(table) for word in caption]
            caption = [word for word in caption if len(word)>1]
            caption = [word for word in caption if word.isalpha()]
            caption_list[i] =  ' '.join(caption)

clean_captions(captions)

In [ ]:
# Create Vocabulary
# extract all unique words from cleaned captions to create a vocabulary

In [ ]:
def to_vocabulary(captions):
    all_captions = []
    for key in captions.keys():
        [all_captions.append(d) for d in captions[key]]
    vocabulary = set()
    for caption in all_captions:
        vocabulary.update(caption.split())
    return vocabulary

vocabulary = to_vocabulary(captions)

In [ ]:
#Saving Cleaned Captions and Vocabulary
#Save the cleaned captions and vocabulary to text files for future reference.

In [ ]:
# Save captions to file
with open('captions.txt', 'w') as captions_file:
    for key, caption_list in captions.items():
        for caption in caption_list:
            captions_file.write(key + '\t' + caption + '\n')

# Save vocabulary to file
with open('vocabulary.txt', 'w') as vocab_file:
    vocab_file.write("\n".join(vocabulary))

In [ ]:
#Preprocessing Images (using pre-trained InceptionV3)
# We use the InceptionV3 model pretrained on ImageNet to extract features from images.
# Images are resized to 299x299 pixels, preprocessed according to InceptionV3 requirements,
# and encoded into a vector of size 2048.

In [ ]:
train_images = glob.glob(os.path.join(dataset_images, '*.jpg'))

# Initialize dictionary to store image encodings
encoding_train = {}

# Loop over each image path and encode it
for img_path in train_images:
    img_name = os.path.basename(img_path)
    encoding_train[img_name] = encode(img_path)

# Function to preprocess an image using InceptionV3
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

# Function to encode an image using InceptionV3 and return its features
def encode(img_path):
    img = preprocess_image(img_path)
    fea_vec = model_new.predict(img)
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1])
    return fea_vec

# Example usage:
start = time()
encoding_train = {}
for img_path in train_images:
    img_name = os.path.basename(img_path)
    encoding_train[img_name] = encode(img_path)
print("Time taken in seconds =", time()-start)

Time taken in seconds = 0.0002663135528564453
